In [ ]:
import numpy as np
import tensorflow as tf
from  tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
import utils.DatasetAPI_to_tensor as VizWiz_Dat
import sklearn.metrics
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
class RecModel():
    def __init__(self):
        self.global_avg = layers.GlobalAveragePooling2D()
        self.dense_1 = layers.Dense(1000, input_shape=(2048,), activation='relu')
        self.dense_2 = layers.Dense(1, input_shape=(1000,), activation='sigmoid')
    
    def build_graph(self, feat):
        self.pred = self.dense_2(self.dense_1(self.global_avg(feat)))
        return self.pred
        
    def comp_loss(self, label):
        loss = K.binary_crossentropy(label, self.pred)
        return loss

In [ ]:
FEAT_SOURCE = 'detectron' # resnet152 or detectron

model = RecModel()

val_ds_iter = VizWiz_Dat.data_iterator('val', feat_source=FEAT_SOURCE, batch_size=256)
test_ds_iter = VizWiz_Dat.data_iterator('test', feat_source=FEAT_SOURCE, batch_size=256)
val_f_map, val_recognizable  = val_ds_iter.get_next()
test_f_map, test_recognizable  = test_ds_iter.get_next()

val_pred = model.build_graph(val_f_map)
test_pred = model.build_graph(test_f_map)

sess = tf.Session()
all_saver = tf.train.Saver() 

In [ ]:
all_saver.restore(sess, './ckpt_rec/{}/{}-ckpt'.format(FEAT_SOURCE, FEAT_SOURCE))
sess.run(test_ds_iter.initializer)
pred_, true_ = sess.run([test_pred, test_recognizable], 
                                feed_dict={keras.backend.learning_phase(): 0})
while True:
    try:
        pred, true = sess.run([test_pred, test_recognizable], 
                                feed_dict={keras.backend.learning_phase(): 0})
        pred_ = np.concatenate((pred_, pred))
        true_ = np.concatenate((true_, true))

    except tf.errors.OutOfRangeError:
        ans_pred_ = 1.0 - pred_
        ans_true_ = 1 - true_
        ap = sklearn.metrics.average_precision_score(ans_true_, ans_pred_)
        break
print(ap)